In [1]:
#import Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
from datetime import datetime

In [2]:
#######################################################
###   EXTRACT - SCRAPE DATA FROM FOOTBALL WEBSITE   ###
#######################################################

def scrape_data():
    # Use the request library to scrape data from the specified link
    web_data = requests.get('https://www.football-data.co.uk/englandm.php')

    #Create a BeautifulSoup object to clean & extract our target data
    soup = BeautifulSoup(web_data.content, 'html.parser')
    links = soup.find_all('a')
    
    """
    Identify links containing the CSV data and save in a list
    Only football data in the CSV formats below are considered:
    - https://www.football-data.co.uk/mmz4281/1920/E0.csv
    - https://www.football-data.co.uk/mmz4281/1920/E2.csv
    - https://www.football-data.co.uk/mmz4281/0203/E1.csv

    """
    # A list to aggregate the matched/desired csv links
    csv_links = []
    for link in links:
        if re.search(r'mmz\d+\/\d+\/(E0|E1|E2)\.csv', str(link)):
            csv_link = re.search(r'mmz\d+\/\d+\/(E0|E1|E2)\.csv', str(link)).group()
            csv_link = 'https://www.football-data.co.uk/'+ csv_link
            csv_links.append(csv_link)
        else:
            continue
    return csv_links

In [3]:
##############################################################################
###       STRUCTURE AND TRANSFORM SCRAPPED DATA TO REQUIRED FORMAT         ###
##############################################################################

#Read data from the csv links and merge into one data file
def extract_data():
    scrapped_links = scrape_data() # Create an object to recieve scrapped data
    datafiles = []
    data_columns = ['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG'] #This is a list of the specific columns of data required
    # Iterate through scrapped csv links, genetate dataframes and combine into a unified dataframe
    for link in scrapped_links:
        csv_data = pd.read_csv(link, usecols=data_columns, sep=',', engine='python', encoding='latin-1')
        datafiles.append(csv_data)
    combined_data = pd.concat(datafiles, axis=0, ignore_index=True) # Merge all data from each csv file into a single dataframe
    # Write data to a csv file
    combined_data.to_csv('football_data.csv', header = ['div','date','home_team','away_team','fthg','ftag'], index = False)

In [4]:
extract_data()

In [5]:
df = pd.read_csv('football_data.csv')

In [6]:
df

,div,date,home_team,away_team,fthg,ftag
0,E0,05/08/2022,Crystal Palace,Arsenal,0.0,2.0
1,E0,06/08/2022,Fulham,Liverpool,2.0,2.0
2,E0,06/08/2022,Bournemouth,Aston Villa,2.0,0.0
3,E0,06/08/2022,Leeds,Wolves,2.0,1.0
4,E0,06/08/2022,Newcastle,Nott'm Forest,2.0,0.0
...,...,...,...,...,...,...
45227,E2,07/05/94,Hartlepool,Plymouth,1.0,8.0
45228,E2,07/05/94,Rotherham,Barnet,1.0,1.0
45229,E2,07/05/94,Stockport,Hull,0.0,0.0
45230,E2,07/05/94,Swansea,Fulham,2.0,1.0
